In [2]:
%%!
pip install --trusted-host pypi.org ipython-sql

In [0]:
%load_ext sql

In [0]:
%sql sqlite://

In [0]:
%%sql
CREATE TABLE CARTOON ( firstname varchar(50), lastname varchar(50));
INSERT INTO CARTOON VALUES('daffy', 'duck');
INSERT INTO CARTOON VALUES('mickey', 'mouse');
SELECT * from CARTOON;


In [0]:
import pandas as pd
BodyTempsData = pd.read_excel('BodyTempsData.xlsx')
BodyTempsData.columns = ['Female', 'Male']

BodyTempsData.head()

In [0]:
%sql DROP TABLE IF EXISTS BodyTempsData

In [0]:
%sql PERSIST BodyTempsData

In [0]:
%sql select count(*) from BodyTempsData where Female_Temp > 98 and Male_Temp > 98

In [0]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,  10))
BodyTempsData.boxplot(['Female','Male'])

In [0]:
import seaborn as sns
sns.boxplot(x="variable", y="value", data=pd.melt(BodyTempsData))


In [0]:
BodyTempsDataMelt=pd.melt(BodyTempsData)
BodyTempsDataMelt.head()

In [0]:
%sql DROP TABLE IF EXISTS BodyTempsDataMelt

In [0]:
%sql PERSIST BodyTempsDataMelt

In [0]:
%sql SELECT variable as Gender, COUNT(*) as NumberOfObservations , MIN(value) as MinTemp, MAX(value) as MaxTemp FROM BodyTempsDataMelt GROUP BY variable ORDER BY value DESC   --ASC